In [1]:
!pip install protobuf==6.31.0

!pip install langchain-google-genai

In [2]:
!python -c "import google.protobuf; print(google.protobuf.__version__)"





6.31.0


In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
import os
os.chdir("../")

In [4]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents 

In [5]:
extracted_data = load_pdf_file(data = 'Data/')
#extracted_data

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("Length of chunks:", len(text_chunks))

Length of chunks: 5859


In [8]:
def downloadHuggingFaceEmbeddings():
    embeddings= HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [9]:
embeddings = downloadHuggingFaceEmbeddings()

/opt/anaconda3/envs/medibot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
query_result = embeddings.embed_query("Hello Man!")
print("Length: ", len(query_result))

Length:  384


In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
from pinecone import Pinecone, ServerlessSpec
import os
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
print(PINECONE_API_KEY)

pcsk_4rYW5e_LNobeLFVTa3yqqycr4YbfMmgHtF9CTMW16DhCs87naJCvFCR2d8JzA9M5VnxGow


In [16]:

pc = Pinecone(api_key= PINECONE_API_KEY)
index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "medibot",
    "metric": "cosine",
    "host": "medibot-z6evhoq.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [14]:
load_dotenv()
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
PINECONE_API_KEY

'pcsk_4rYW5e_LNobeLFVTa3yqqycr4YbfMmgHtF9CTMW16DhCs87naJCvFCR2d8JzA9M5VnxGow'

In [17]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents= text_chunks,
    index_name= index_name,
    embedding=embeddings, 
)

In [18]:
#Load Existing Index
docsearch= PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

docsearch

In [19]:
retriever = docsearch.as_retriever(search_type='similarity', search_kwargs={'k':3})
retriever_docs = retriever.invoke("What is cancer?")
retriever_docs

[Document(id='860f49e7-3317-4a4e-81b6-b1c72d690a6a', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 184.0, 'page_label': '185', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='KEY TERMS\nBiopsy—A procedure in which a small piece of\nbody tissue is removed and examined under a\nmicroscope for cancer.\nChemotherapy —A cancer treatment in which\ndrugs delivered into the blood stream kill cancer\ncells or make them more vulnerable to radiation\ntherapy.\nHuman papillomavirus (HPV)—A virus with many\nsubtypes, some of which cause cell changes that\nincrease the risk of certain cancers.\nHuman immunodeficiency virus (HIV)—The virus\nthat causes acquired immune deficiency syn-'),
 Document(id='1aa2dd44-97e3-4b81-af75-fafe718b94c2', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 59

In [20]:
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model ='gemini-2.0-flash',temperature=0.4, max_tokens=60)

In [22]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [23]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
     "Use the following pieces of retrieved context to answer "
     "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
     "\n\n"
     "{context}"
 )

prompt = ChatPromptTemplate.from_messages(
     [
        ("system", system_prompt),
         ("human", "{input}"),
     ]
 )


In [24]:
#Initialize RAG Chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [26]:
response = rag_chain.invoke({"input": "What is Cancer?"})
print(response["answer"])

Cancer is caused by the development of malignant cells. Cancer cells are characterized by uncontrolled division. In breast cancer, the malignant cells originate in the lining of the milk glands or ducts of the breast.
